<a href="https://colab.research.google.com/github/Nihala-RI/Recommendation-systems/blob/demo/Collaberative_filtering_using_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
movies_df=pd.read_csv('/content/movies.csv',usecols=['movieId','title'],dtype={'movieId':'int32','title':'str'})
ratings_df=pd.read_csv('/content/ratings.csv',usecols=['userId','movieId','rating'],dtype={'userId':'int32','movieId':'int32','rating':'float32'})

In [3]:
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [4]:
ratings_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [5]:
df=pd.merge(movies_df,ratings_df,on='movieId',how='inner')

In [6]:
df.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [7]:
combine_movie_rating = df.dropna(axis = 0, subset = ['title'])
movie_ratingCount = (combine_movie_rating.groupby('title')['rating'].count().
                     reset_index().rename(columns={'rating':'TotalRatingCount'}))

In [8]:
combine_movie_rating

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5
...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),184,4.0
100832,193583,No Game No Life: Zero (2017),184,3.5
100833,193585,Flint (2017),184,3.5
100834,193587,Bungo Stray Dogs: Dead Apple (2018),184,3.5


In [9]:
movie_ratingCount

,title,TotalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2
...,...,...
9714,eXistenZ (1999),22
9715,xXx (2002),24
9716,xXx: State of the Union (2005),5
9717,¡Three Amigos! (1986),26


In [10]:
rating_with_totalRatingCount=pd.merge(combine_movie_rating,movie_ratingCount,left_on='title',right_on='title',how='inner')
rating_with_totalRatingCount

,movieId,title,userId,rating,TotalRatingCount
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215
...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),184,4.0,1
100832,193583,No Game No Life: Zero (2017),184,3.5,1
100833,193585,Flint (2017),184,3.5,1
100834,193587,Bungo Stray Dogs: Dead Apple (2018),184,3.5,1


In [11]:
pd.set_option('display.float_format',lambda x: '%.3f' % x)
print(movie_ratingCount['TotalRatingCount'].describe())

count   9719.000
mean      10.375
std       22.406
min        1.000
25%        1.000
50%        3.000
75%        9.000
max      329.000
Name: TotalRatingCount, dtype: float64


In [12]:
popularity_threshold = 50
rating_popularmovie=rating_with_totalRatingCount.query('TotalRatingCount>= @popularity_threshold')
rating_popularmovie.head()

,movieId,title,userId,rating,TotalRatingCount
0,1,Toy Story (1995),1,4.000,215
1,1,Toy Story (1995),5,4.000,215
2,1,Toy Story (1995),7,4.500,215
3,1,Toy Story (1995),15,2.500,215
4,1,Toy Story (1995),17,4.500,215


In [13]:
rating_popularmovie.shape

(41362, 5)

In [14]:
movie_features_df=rating_popularmovie.pivot_table(index='title',columns='userId',values='rating').fillna(0)

In [15]:
movie_features_df

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,3.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000
12 Angry Men (1957),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
28 Days Later (2002),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.500,0.000,5.000
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X2: X-Men United (2003),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,4.000
You've Got Mail (1998),0.000,0.000,0.500,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,2.000,0.000,0.000,3.500,0.000,0.000,0.000,0.000
Young Frankenstein (1974),5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,3.500,0.000,0.000,0.000,0.000


In [16]:
from scipy.sparse import csr_matrix
movie_features_df_matrix=csr_matrix(movie_features_df.values)
movie_features_df_matrix

<450x606 sparse matrix of type '<class 'numpy.float32'>'
	with 41360 stored elements in Compressed Sparse Row format>

In [17]:
from sklearn.neighbors import NearestNeighbors
model_knn=NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [18]:
movie_features_df.shape

(450, 606)

In [20]:
query_index=np.random.choice(movie_features_df.shape[0])
print(query_index)
distances,indices=model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1,-1),n_neighbors=6)

97


In [ ]:
movie_features_df.iloc[query_index]

In [ ]:
movie_features_df.iloc[query_index,:]

In [ ]:
movie_features_df.iloc[query_index,:].values

In [ ]:
movie_features_df.iloc[query_index,:].values.reshape(1,-1)

In [ ]:
distances

In [ ]:
len(distances)

In [ ]:
distances.flatten()

In [ ]:
len(distances.flatten())

In [ ]:
indices

In [ ]:
len(indices.flatten())

In [ ]:
indices[0,3]

In [ ]:
indices.flatten()[3]

In [ ]:
movie_features_df.index[97]

In [47]:
for i in range(0,len(distances.flatten())):
  if i==0:
    print("Recommendations for {}:\n".format(movie_features_df.index[query_index]))
  else:
    print("{}:{},with distance of {}".format(i,movie_features_df.index[indices.flatten()[i]],distances.flatten()[i]))

Recommendations for Clear and Present Danger (1994):

1:Crimson Tide (1995),with distance of 0.3035197854042053
2:Fugitive, The (1993),with distance of 0.32463133335113525
3:Firm, The (1993),with distance of 0.33566951751708984
4:Cliffhanger (1993),with distance of 0.3399702310562134
5:True Lies (1994),with distance of 0.34541380405426025
